In [1]:
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import polars as pl
from skimage.feature import hog

In [ ]:
def restore_image_from_list(width: int, height: int, image: list) -> np.array:
    return np.array(image).reshape((height, width, 3))


def create_hog_features(image):
    image = cv2.cvt
    features, hog_image = hog(
        image,
        block_norm="L2-Hys",
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        visualize=True,
    )
    return features, hog_image

In [3]:
# Read the parquet file, this takes a while. Leave it here
dft = pl.read_parquet("Train.parquet", use_pyarrow=True, memory_map=True)

In [ ]:
samples = dft.sample(100, with_replacement=False)